In [5]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

Final_file_path = "cleaned_file.csv"
data = pd.read_csv(Final_file_path, low_memory=False)

In [10]:
# Print alle unikke værdier i 'type' kolonnen
print(data['type'].unique())

# Fjern rækker hvor 'type' kolonnen har bestemte værdier
data = data[~data['type'].isin(["['satir']", "['conspiraci']", "['junksci']", "['unreli']", "['rumor']", "['unknown']", "['nan']", "['hate']", "['num', 'date', 'num', 'num', 'num']", "[]"])]
data = data.dropna(subset=["type"])

# Print alle unikke værdier i 'type' kolonnen efter fjernelse
print(data['type'].unique())

# Definér mapping
label_mapping = {
    "political": "True",
    "clickbait": "True",
    "reliable": "True",
    "fake": "Fake",
    "bias": "Fake"
}

# Opret label-kolonnen først
data["label"] = data["type"].map(label_mapping)

# Konverter 'True' til 1 og 'Fake' til 0
data["label"] = data["label"].map({"True": 1, "Fake": 0})


# Definér y (målvariabel)
global y
y = data["label"].dropna().astype(int)

# Tjek om det virker
print(y.unique())  # Skal vise [1 0]


["['polit']" "['fake']" "['reliabl']" "['bia']" '[]' "['clickbait']"]
["['polit']" "['fake']" "['reliabl']" "['bia']" "['clickbait']"]
[]


In [21]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score
import numpy as np

# Load CSV
data = pd.read_csv("cleaned_file.csv")

#Fjerner class with less than 1 sample
min_samples = 2
y_counts = pd.Series(y).value_counts()
valid_classes = y_counts[y_counts >= min_samples].index

#Vectorize text data
vectorizer = TfidfVectorizer(max_features=10000, binary=True)
X = vectorizer.fit_transform(data["content"])

# Filter dataset
X_filtered = X[np.isin(y, valid_classes)]
y_filtered = y[np.isin(y, valid_classes)]

#Fjerner class with less than 1 sample
min_samples = 2
y_counts = pd.Series(y).value_counts()
valid_classes = y_counts[y_counts >= min_samples].index

# Ensure text data is string type
data["content"] = data["content"].fillna("").astype(str)
data["type"] = data["type"].fillna("").astype(str)



# Encoding the labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(data["type"])

# Splitting dataset into train, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Implementing logistic regression
clf = LogisticRegression(max_iter=500, random_state=42, solver="saga", n_jobs=-1)
clf.fit(X_train, y_train)

# Evaluating the model
y_pred = clf.predict(X_test)
f1 = f1_score(y_test, y_pred, average="weighted")  # Use 'weighted' for multi-class classification

# Display results
print(f"F1-score: {f1:.4f}")
print("Hyperparameters for Logistic Regression:")
print(f"Max Iterations: {clf.max_iter}")
print(f"Solver: {clf.solver}")
print(f"Random State: {clf.random_state}")


C:\Users\emilv\AppData\Local\Temp\ipykernel_12108\1780782999.py:10: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("cleaned_file.csv")


F1-score: 0.7543
Hyperparameters for Logistic Regression:
Max Iterations: 500
Solver: saga
Random State: 42
